In [1]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
from langchain.callbacks import get_openai_callback

text = """
### 質問

楽曲『約束はいらない』でデビューした、声優は誰？

### コンテキスト

(0.21018645, '約束はいらない', '「約束はいらない」(やくそくはいらない)は、坂本真綾のデビューシングル。') (0.24241784, '約束はいらない', '坂本真綾の歌手デビュー作品。当時坂本はまだ無名の声優であったが、同曲がテーマソングとなったアニメ『天空のエスカフローネ』とともに知名度を上げることとなる。後に「指輪」が同アニメの劇場版映画の主題歌とな') (0.2441598, '坂本真綾', '本格的な歌手活動は、1996年にテレビアニメ『天空のエスカフローネ』へ出演したことがきっかけで始めており、同作のオープニングテーマソングである「約束はいらない」(岩里祐穂作詞、菅野よう子作曲)をシング') (0.26043588, 'Suara', '大阪府立豊中高等学校を経て大阪外国語大学でインドネシア語を専攻中にバンド・ユニットを組んで音楽活動を始めた。普段はお笑い番組が大好きなこともあってよく喋るほうだが、東京の仕事で標準語の喋りをする時は、') (0.2690969, '早見沙織', '声優デビュー時より、数多くの主題歌やキャラクターソングを担当し、バラードからポップス、ヒットソングのカバー曲や英語の楽曲など、様々な曲を歌いこなす。2009年には吉田仁美とのユニット「blue dro')

### 回答: 
"""
with get_openai_callback() as cb:
    res = llm.generate([text], max_tokens=24, temperature=0.0)

In [2]:
res.generations[0][0].text.strip()

'坂本真綾'

In [3]:
cb

Tokens Used: 633
	Prompt Tokens: 625
	Completion Tokens: 8
Successful Requests: 1
Total Cost (USD): $0.0009534999999999999

In [4]:
RESPONSE_MESSAGE = "Answer"
RESPONSE_PROMPT = f"### {RESPONSE_MESSAGE}: \n"


def build_prompt(
    user_message: str,
    inputs: str | None = "",
    separator: str = "\n\n### ",
    response_message: str = RESPONSE_MESSAGE,
) -> str:
    prompt = ""
    roles = ["Instruction", response_message]
    messages = [": \n" + user_message, ": \n"]

    if inputs:
        roles = ["Instruction", "Context", response_message]
        messages = [": \n" + user_message, ": \n" + inputs, ": \n"]

    for role, message in zip(roles, messages):
        prompt += separator + role + message
    return prompt

In [5]:
import pandas as pd
import datasets

ds = datasets.load_dataset("hotchpotch/jaqket_v1_qa_wikija_context")  # type: ignore
valid_ds = ds["validation"]  # type: ignore
valid_df = valid_ds.data.to_pandas()  # type: ignore
# context は list なので、 "\n" で結合する
valid_df["context"] = valid_df["context"].apply(lambda x: "\n".join(x) + "\n")
valid_df.head(1)

/home/yu1/miniconda3/envs/llm-sc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,qid,question,answer,context,answers,competition,timestamp,section,number,original_question,original_answer,original_additional_info
0,QA20QBIK-0912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャ...,[アルジャーノンに花束を],第1回AI王,2020/01/27,開発データ問題 (dev1),912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,


In [6]:
def qa_prompt(question, context, build_prompt_fn=build_prompt):
    prompt = build_prompt_fn(question, context)
    return prompt

In [7]:
# valid_df での正解率を測る
from tqdm import tqdm

# valid_df = valid_df.head(10)

valid_df = valid_df.reset_index(drop=True)

HINT_MESSAGE = " Please respond with reference to the following Context."

with get_openai_callback() as cb:
    for i in tqdm(range(len(valid_df))):
        prompt = qa_prompt(
            valid_df["question"][i] + HINT_MESSAGE, valid_df["context"][i]
        )
        if i == 0:
            print(prompt)
        answer = valid_df["answer"]
        res = llm.generate([prompt], max_tokens=24, temperature=0.0)
        valid_df.loc[i, "pred"] = res.generations[0][0].text.strip()

  0%|          | 0/980 [00:00<?, ?it/s]



### Instruction: 
手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイスの小説は何? Please respond with reference to the following Context.

### Context: 
ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャーリイが実験的手術によって天才となり、短期間でその効果が消えてしまった経過を報告書形式で書いたものである。中編が1959年4月号のF&SF誌に掲載され、1966年に長編化された。この小説は何度か他のメディアに採用されており、1968年の映画『まごころを君に』(出演: クリフ・ロバートソン、クレア・ブルーム)、2002年の日本のテレビドラマ『アルジャーノンに花束を』(出演: ユースケ・サンタマリア、菅野美穂)、などがある。クリフ・ロバートソンはこの作品でアカデミー主演男優賞を受賞している。また、キイスは中編版で1959年にヒューゴー賞、長編版で1966年にネビュラ賞を受賞した。また2015年にも日本のテレビドラマ『アルジャーノンに花束を』(出演:山下智久、窪田正孝など)で新版長編も放送された。
ダニエル・キイス ニューヨーク州ブルックリン区生まれ。17歳のとき U.S. Maritime Service に就職し船員となった。その後しばらくして、働きながら心理学や文学に興味を抱くようになりブルックリンカレッジで心理学の学士号を取得し、一時期ファッション写真のスタジオで働き、その後ニューヨークの高校で国語教師を務めつつ、定時制で英米文学を学び、週末には小説を書いていた。最終的に英米文学の修士号を得ている。1950年代初め、パルプ雑誌『マーヴェル・サイエンス・ストーリーズ』の編集者となった。この雑誌の出版者であるマーティン・グッドマン(英語版)はアメリカン・コミックスも手がけており、マーベル・コミックの前身となったタイムリー・コミック(1940年代)やアトラス・コミック(1950年代)を出版していた。
アルジャーノンに花束を 『アルジャーノンに花束を』(アルジャーノンにはなたばを、Flowers for Algernon)は、アメリカ合衆国の作家ダニエル・キイスによるSF小説。1959年に中編小説

100%|██████████| 980/980 [06:45<00:00,  2.41it/s]


In [8]:
cb

Tokens Used: 947778
	Prompt Tokens: 939095
	Completion Tokens: 8683
Successful Requests: 980
Total Cost (USD): $1.4260084999999998

In [9]:
# 完全一致の正解率を表示
valid_df["is_correct"] = valid_df["answer"] == valid_df["pred"]
valid_df["is_correct"].mean()

0.5948979591836735

In [10]:
# 間違ったものだけを表示
incorrect_df = valid_df[["question", "answer", "pred", "is_correct"]][
    valid_df["is_correct"] == False
]

In [11]:
# 部分一致の正解率を表示
valid_df["is_correct"] = valid_df.apply(lambda x: x["answer"] in x["pred"], axis=1)
valid_df["is_correct"].mean()

0.7989795918367347

In [12]:
valid_df[["question", "answer", "pred", "is_correct"]].head(100)
# 間違ったものだけを表示
valid_df[["question", "answer", "pred", "is_correct"]][valid_df["is_correct"] == False]

,question,answer,pred,is_correct
8,ラテン語では「アーリア・ヤクタ・エスト」という、カエサルがルビコン川を渡る際に言ったとされる...,賽は投げられた,「アーリア・ヤクタ・エスト」,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,「つけあげ」は、鹿児島県の郷土料理,False
14,飛行機の操縦士としての体験を元に小説『夜間飛行』や童話『星の王子様』などの作品を残したフラン...,アントワーヌ・ド・サン=テグジュペリ,アントワーヌ・ド・サン＝テグジュペリ (Antoine de Saint,False
22,日本銀行法第8条で、「一億円とする」と定められているのは、日本銀行に関するどんなものについて...,資本金,日本銀行法第8条で、「一億円とする」と定められ,False
26,水揚げからの鮮度の低下が早く「生き腐れ」とも呼ばれる、漢字で「鯖」と書く魚は何?,サバ,鯖,False
...,...,...,...,...
958,弦を地面と水平に張っている、コンサートなどに使用される大型のピアノを何というでしょう?,グランドピアノ,コンサート・グランド・ピアノ,False
964,旅客機の座席で、ファーストクラスとエコノミークラスの中間にあたるものを一般に何クラスというで...,ビジネスクラス,プレミアム・エコノミー,False
967,2人で行う花札の遊び方で、「五光」や「猪鹿蝶」といった役があるのは何?,こいこい,「五光」や「猪鹿蝶」といった役があ,False
969,1932年に満州事変の調査のため、国際連盟により派遣された調査団のことを、団長を務めたイギリ...,リットン調査団,リットン卿,False


In [13]:
print(
    valid_df[["question", "answer", "pred", "is_correct"]][
        valid_df["is_correct"] == False
    ].to_csv()
)

,question,answer,pred,is_correct
8,ラテン語では「アーリア・ヤクタ・エスト」という、カエサルがルビコン川を渡る際に言ったとされる有名な言葉は何でしょう?,賽は投げられた,「アーリア・ヤクタ・エスト」,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,「つけあげ」は、鹿児島県の郷土料理,False
14,飛行機の操縦士としての体験を元に小説『夜間飛行』や童話『星の王子様』などの作品を残したフランスの作家は誰でしょう?,アントワーヌ・ド・サン=テグジュペリ,アントワーヌ・ド・サン＝テグジュペリ (Antoine de Saint,False
22,日本銀行法第8条で、「一億円とする」と定められているのは、日本銀行に関するどんなものについてでしょう?,資本金,日本銀行法第8条で、「一億円とする」と定められ,False
26,水揚げからの鮮度の低下が早く「生き腐れ」とも呼ばれる、漢字で「鯖」と書く魚は何?,サバ,鯖,False
28,鎌倉幕府や室町幕府が荘園を管理支配するために設置した役職で、道理の通じない相手の例えとして「泣く子とコレには勝てぬ」ともいわれるのは何でしょう?,地頭,守護使不入(しゅごしふにゅう),False
29,人間の太ももの裏側にある、大腿四頭筋と対になる大腿二頭筋・半膜様筋・半腱様筋などを総称して、「もも肉のひも」という意味の英語で何という?,ハムストリング,hamstrings,False
40,サッカーのスコットランドリーグで、レンジャーズやセルティックが本拠地とする都市はどこ?,グラスゴー,レンジャーズやセルティックが本拠地とする都市はグラ,False
53,化粧品メーカー「資生堂」の名前の由来となった一節が登場する、中国の四書五経のひとつは何?,易経,書経,False
59,イタリア北部に位置するピエモンテ州の州都で、2006年には冬のオリンピックの開催地となったのはどこでしょう?,トリノ,プラジェラート,False
72,兵庫県の「岩津」、埼玉県の「深谷」、群馬県の「下仁田」といえばどんな野菜のブランドでしょう?,ネギ,岩津ねぎ、深谷ねぎ、下仁田,False
80,柔道8段の段位を持ち、ヨーロッパ柔道連盟や国際柔道連盟の名誉会長も務めている、ロシアの政治家は誰?,ウラジー